In [1]:
import cv2
import imutils
from imutils import face_utils
import dlib
import numpy as np
import webcolors
import math

# Initialisiere eine Flagge
flag = 0

# Lade den dlib Gesichtsdetektor und den Prädiktor für die Gesichtspunkte
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("Utils/shape_predictor_68_face_landmarks.dat")

# Lese das Bild in RGB-Format ein
img_rgb = cv2.imread('Utils/Maurice.jpg')

# Bestimme die Indizes der Gesichtspunkte für das linke und rechte Auge
(left_Start, left_End) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_Start, right_End) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# Funktion zur Bestimmung des Farbnamen aus den RGB-Werten
def find_color(requested_colour):
    min_colours = {}
    for name, key in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(name)
        bd = (b_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        rd = (r_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = key
    closest_name = min_colours[min(min_colours.keys())]
    return closest_name

# Konvertiere das Bild in Graustufen
gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

# Erkenne Gesichter im Graustufenbild
dlib_faces = detector(gray, 0)

# Verarbeite jedes erkannte Gesicht
for face in dlib_faces:
    eyes = []  # Liste zur Speicherung der Augen
    
    # Konvertiere das dlib Rechteck in ein Bounding Box
    (x, y, w, h) = face_utils.rect_to_bb(face)
    
    # Zeichne ein Rechteck um das Gesicht
    cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (255, 0, 0), 1)
    
    # Bestimme die Gesichtspunkte
    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)
    
    # Extrahiere die Punkte für die Augen
    leftEye = shape[left_Start:left_End]
    rightEye = shape[right_Start:right_End]
    
    # Füge die Augenpunkte zur Liste hinzu
    eyes.append(leftEye)
    eyes.append(rightEye)
    
    # Verarbeite die Augenpunkte
    for index, eye in enumerate(eyes):
        flag += 1
        left_side_eye = eye[0]  # Linker Rand des Auges
        right_side_eye = eye[3]  # Rechter Rand des Auges
        top_side_eye = eye[1]  # Oberer Rand des Auges
        bottom_side_eye = eye[4]  # Unterer Rand des Auges

        # Berechne die Breite und Höhe des Auges
        eye_width = right_side_eye[0] - left_side_eye[0]
        eye_height = bottom_side_eye[1] - top_side_eye[1]

        # Definiere ein Rechteck um die Augenregion
        eye_x1 = int(left_side_eye[0] - 0 * eye_width)
        eye_x2 = int(right_side_eye[0] + 0 * eye_width)
        eye_y1 = int(top_side_eye[1] - 1 * eye_height)
        eye_y2 = int(bottom_side_eye[1] + 0.75 * eye_height)

        # Extrahiere die Region um das Auge (ROI)
        roi_eye = img_rgb[eye_y1:eye_y2, eye_x1:eye_x2]
        
        # Verlasse die Schleife nach dem ersten Auge
        if flag == 1:
            break

# Bestimme die Größe der Augenregion
x = roi_eye.shape
row = x[0]
col = x[1]

# Wähle die RGB-Werte aus dem Bereich direkt unter der Pupille
array1 = roi_eye[(row // 2):(row // 2) + 1, int((col // 3) + 2):int((col // 3)) + 9]
array1 = array1[0][5]
array1 = tuple(array1)

# Finde den Farbnamen für die ausgewählten RGB-Werte
color_name = find_color(array1)
print(color_name)

# Erstelle ein Bild zur Anzeige der erkannten Farbe
detected_color = np.zeros((100, 300, 3), dtype=np.uint8)
detected_color[:] = array1[::1]

# Füge den Farbnamen zum Bild hinzu
cv2.putText(detected_color, color_name, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Zeige das Bild mit der erkannten Farbe an
cv2.imshow("Detected Color", detected_color)

# Markiere den Bereich um die Pupille in der Augenregion
roi_top = (row // 2)
roi_bottom = (row // 2) + 1
roi_left = (col // 3) + 2
roi_right = (col // 3) + 9
new_rgb = (0, 0, 255)
roi_eye[roi_top:roi_bottom, roi_left:roi_right] = new_rgb

# Zeige die Augenregion mit der markierten Farbe an
cv2.imshow("frame", roi_eye)
cv2.waitKey(0)
cv2.destroyAllWindows()

midnightblue
